In [95]:
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By

In [96]:
# The jobs page automatically defaults to your location. Therefore, it turns up 0 results.
# The below code will help with making the jobs page not default to a specific locaion.

driver = webdriver.Chrome()
driver.implicitly_wait(10)

url = 'https://careers.walmart.com/'
driver.get(url)

job_search = driver.find_element(By.XPATH, '//*[@id="search"]')
job_search.send_keys('technology')

search_button = driver.find_element(By.XPATH, '//*[@id="location"]')


search_button.click()

In [97]:
#job_num = []
TITLE = []
LINK = []

y = 1
i = 1

while y != 0:  
#while i < 3:  
    
    try:
        
        url = 'https://careers.walmart.com/results?q=&page='+str(i)+'&sort=rank&jobCategory=00000161-7bad-da32-a37b-fbef5e390000,00000161-7bf4-da32-a37b-fbf7c59e0000,00000161-7bff-da32-a37b-fbffc8c10000,00000161-8bd0-d3dd-a1fd-bbd0febc0000,00000161-8be6-da32-a37b-cbe70c150000&jobSubCategory=0000015a-a577-de75-a9ff-bdff284e0000&expand=department,0000015e-b97d-d143-af5e-bd7da8ca0000,00000161-8be6-da32-a37b-cbe70c150000,brand,type,rate&type=jobs'
        
        driver.get(url) 
    
        for j in range(1,26): 

            job = driver.find_element(By.XPATH, '//*[@id="search-results"]/li['+str(j)+']/div[1]/h4/a')
            #print(job)

            try:
                TITLE.append(job.text)
            except:
                TITLE.append('')

            try:
                LINK.append(job.get_attribute('href'))
            except:
                LINK.append('')     
        
    except:
        y -= 1
    
    #print('page', i)
    
    i += 1
    
driver.close()    
print(len(TITLE), len(LINK)) 


355 355


In [98]:
print(TITLE[1:10])

['Senior Manager II, Technical Program Management', '(USA) Senior Director II, Technical Program Management', 'Staff Technical Program Manager- Information Security Business', 'Staff Technical Program Manager', 'Director, Technology Operations - Post Transaction Fulfilment Experience Remediation & Mitigation - Business Strategy', 'Director, Technology Operations - Post Transaction Fulfilment Clarity & Control Experience - Business Strategy', '(USA) Senior Fuel Operations Manager - WM NBD', '(USA) Senior Manager II, Technology Operations, SOC', '(USA) Senior Manager II, Technology Operations']


In [99]:
print(LINK[1:5])

['https://careers.walmart.com/us/jobs/WD1412797-senior-manager-ii-technical-program-management', 'https://careers.walmart.com/us/jobs/WD1416415-usa-senior-director-ii-technical-program-management', 'https://careers.walmart.com/us/jobs/WD1391299-staff-technical-program-manager-information-security-business', 'https://careers.walmart.com/us/jobs/WD1397133-staff-technical-program-manager']


In [100]:
df = pd.DataFrame(zip(TITLE, LINK))
df['QUALIFICATIONS'] = np.nan


In [101]:
df.columns = ['TITLE', 'LINK', 'QUALIFICATIONS']
df.head()

,TITLE,LINK,QUALIFICATIONS
0,(USA) Staff Technical Program Manager - Tech,https://careers.walmart.com/us/jobs/WD1379878-...,NaN
1,"Senior Manager II, Technical Program Management",https://careers.walmart.com/us/jobs/WD1412797-...,NaN
2,"(USA) Senior Director II, Technical Program Ma...",https://careers.walmart.com/us/jobs/WD1416415-...,NaN
3,Staff Technical Program Manager- Information S...,https://careers.walmart.com/us/jobs/WD1391299-...,NaN
4,Staff Technical Program Manager,https://careers.walmart.com/us/jobs/WD1397133-...,NaN


In [102]:
df = df.drop_duplicates(subset=['TITLE'])
len(df)

218

In [103]:
df.to_csv('walmart_jobs_title_n_link_updated012723.csv')

In [104]:
driver = webdriver.Chrome()

for i in range(len(df['LINK'])):

    try:
        
        url = (df['LINK'][i])

        driver.get(url)

        desc = driver.find_element(By.XPATH, '/html/body/main/section[3]/div/div[2]')
        

        df['QUALIFICATIONS'][i] = desc.text   
         
    except:
        df['QUALIFICATIONS'][i] = np.nan

driver.close()
        


C:\Users\meske\AppData\Local\Temp/ipykernel_24032/3919239858.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['QUALIFICATIONS'][i] = desc.text


In [105]:
df.isna().sum()

TITLE              0
LINK               0
QUALIFICATIONS    78
dtype: int64

In [106]:
df = df.drop_duplicates(subset=['TITLE', 'LINK', 'QUALIFICATIONS'])
df['QUALIFICATIONS'] = df['QUALIFICATIONS'].str.lower()
df = df.dropna()
df['COMPANY'] = 'Walmart'
df = df.reset_index(drop = True)
len(df)

140

In [107]:
for i in range(len(df['QUALIFICATIONS'])):

    try:

        desc = df['QUALIFICATIONS'][i]
        mid = desc.index('minimum qualifications')+22
        desc = desc[mid:]
        df['QUALIFICATIONS'][i] = desc

    except:
        pass


In [108]:
# removing unnecessary text

remove1 = '...\noutlined below are the required minimum qualifications for this position. if none are listed, there are no minimum qualifications.'
remove2 = '...\noutlined below are the optional preferred qualifications for this position. if none are listed, there are no preferred qualifications.'

In [109]:
for i in range(len(df['QUALIFICATIONS'])):

    try:

        df['QUALIFICATIONS'][i] = df['QUALIFICATIONS'][i].replace(remove1, '')
        df['QUALIFICATIONS'][i] = df['QUALIFICATIONS'][i].replace(remove2, '')
        df['QUALIFICATIONS'][i] = df['QUALIFICATIONS'][i].replace('\n', '')
        df['QUALIFICATIONS'][i] = df['QUALIFICATIONS'][i].replace('• ', '')
        
    except:
        pass

In [110]:
for i in range(len(df['QUALIFICATIONS'])):

    try:

        desc = df['QUALIFICATIONS'][i]
        mid = desc.index('primary location')
        desc = desc[:mid]
        df['QUALIFICATIONS'][i] = desc      
        df['QUALIFICATIONS'][i] = df['QUALIFICATIONS'][i].strip()

    except:
        pass


In [111]:
df['DESCRIPTION'] = df['QUALIFICATIONS']
df = df[['COMPANY', 'TITLE', 'QUALIFICATIONS', 'LINK', 'DESCRIPTION']]
df.head()

,COMPANY,TITLE,QUALIFICATIONS,LINK,DESCRIPTION
0,Walmart,(USA) Staff Technical Program Manager - Tech,minimum qualifications: bachelor of science an...,https://careers.walmart.com/us/jobs/WD1379878-...,minimum qualifications: bachelor of science an...
1,Walmart,"Senior Manager II, Technical Program Management",option 1: bachelor’s degree in computer scienc...,https://careers.walmart.com/us/jobs/WD1412797-...,option 1: bachelor’s degree in computer scienc...
2,Walmart,"(USA) Senior Director II, Technical Program Ma...",option 1: bachelor’s degree in computer scienc...,https://careers.walmart.com/us/jobs/WD1416415-...,option 1: bachelor’s degree in computer scienc...
3,Walmart,Staff Technical Program Manager- Information S...,option 1: bachelor’s degree in computer scienc...,https://careers.walmart.com/us/jobs/WD1391299-...,option 1: bachelor’s degree in computer scienc...
4,Walmart,Staff Technical Program Manager,option 1: bachelor’s degree in computer scienc...,https://careers.walmart.com/us/jobs/WD1397133-...,option 1: bachelor’s degree in computer scienc...


In [94]:
df.to_csv('walmart_technology_jobs.csv')